In [1]:
import pandas as pd
import re

In [5]:
dict_df = pd.read_csv('data/lexicon.csv')
dict_list = dict_df.word.tolist()
dict_df = dict_df.set_index('word')

sentences_df = pd.read_csv('data/sentences.csv')

In [11]:
pos = open('data/lexicon_based_annotated.txt', 'w', encoding='utf8')

count=1
for i, row in sentences_df.iterrows():
    sentence = row['sentence']
    sent_id = str(row['id'])
    
    sentence = sentence.replace('\n','')
    sentence = sentence.replace('\"', ' ')
    sentence = sentence.replace('،', ' ، ') 
    sentence = re.sub(' +', ' ', sentence)
    sentence = sentence.strip()

    segments = sentence.split(' ')
    total_segments = len(segments)

    tagged_sent = ''
    segmented = ''
    tagged_sent_sql = []
    skip = 0
    for index, segment in enumerate(segments):  
        if(skip>0):
            skip = skip - 1
            continue

        word = ''
        tag = 'NULL'
        row = pd.Series(dtype='float64')

        word1 = segments[index]
        word2 = ' '.join(segments[index:index+2])
        word3 = ' '.join(segments[index:index+3])
        word4 = ' '.join(segments[index:index+4])
        word5 = ' '.join(segments[index:index+5])
        word1 = word1.strip()
        word4 = word4.strip()
        word5 = word5.strip()
        
        if(word5 in dict_list):
            word = word5
            row = dict_df.loc[word]
            skip = 4
        elif(word4 in dict_list):
            word = word4
            row = dict_df.loc[word]
            skip = 3
        elif(word3 in dict_list):
            word = word3
            row = dict_df.loc[word]
            skip = 2
        elif(word2 in dict_list):
            word = word2
            row = dict_df.loc[word]
            skip = 1
        elif(word1 in dict_list):
            word = word1
            row = dict_df.loc[word]
        else: word = word1

        if(not row.empty): tag = str(row.tag)
        word = word.strip()
        
        if(word == '،'): tag = 'PU'
        if(word.isdigit()): tag = 'NB'
            
        if(tag == 'NULL'): unknown.write(sent_id + '\t' + word + '\t' + sentence + '\n')
        
        tag = tag.strip('. ')
        tag = re.sub('\.+', '.', tag)
        tag = tag.upper()
        tag = tag.replace('P.III', 'P.iii')
        tag = tag.replace('P.II', 'P.ii')
        tag = tag.replace('P.I', 'P.i')
        
        word_tag_pair_sql = [word, tag]
        tagged_sent_sql.append(word_tag_pair_sql)
        
        tagged_sent = tagged_sent + word +  '_' + tag.strip() + '|'
        segmented = segmented + word +  'B'
        
    if(not 'NULL' in str(tagged_sent)):
        print('Complete Cont: ', count)
        count+=1
        pos.write(sent_id+'\t'+tagged_sent.strip(' |')+'\n')     

pos.close()